# I - Problem 1 (TD 1)

## 1 - Linear version

In [1]:
from gurobipy import Model, GRB

In [2]:
model = Model("Unicorn SAT")
model.setParam("OutputFlag", 0)

# Définir les variables booléennes (1: vrai, 0: faux)
mythique = model.addVar(vtype=GRB.BINARY, name="mythique")
mortelle = model.addVar(vtype=GRB.BINARY, name="mortelle")
mammifere = model.addVar(vtype=GRB.BINARY, name="mammifere")
corne = model.addVar(vtype=GRB.BINARY, name="corne")
magique = model.addVar(vtype=GRB.BINARY, name="magique")

# Ajouter les contraintes de la base de connaissances
model.addConstr((1 - mythique) + (1 - mortelle) >= 1, "C1")  # -1 -2 0
model.addConstr(mythique + mortelle >= 1, "C2")              # 1 2 0
model.addConstr(mythique + mammifere >= 1, "C3")            # 1 3 0
model.addConstr((1 - mammifere) + corne >= 1, "C4")         # -3 4 0
model.addConstr(mortelle + corne >= 1, "C5")                # 2 4 0
model.addConstr((1 - corne) + magique >= 1, "C6")           # -4 5 0

# Optimiser le modèle
model.optimize()

# Vérifier la cohérence de la base de connaissances
if model.status == GRB.OPTIMAL:
    print("La base de connaissances est cohérente.")
    print("Un modèle trouvé (monde peuplé de licornes) :")
    print(f"Mythique: {int(mythique.x)}, Mortelle: {int(mortelle.x)}, Mammifère: {int(mammifere.x)}, Corne: {int(corne.x)}, Magique: {int(magique.x)}")

    # Vérifier si la licorne a une corne
    corne.setAttr(GRB.Attr.LB, 1)  # Fixer la variable "corne" à 1
    model.optimize()
    has_horn = (model.status == GRB.OPTIMAL)

    # Vérifier si la licorne n'a pas de corne
    corne.setAttr(GRB.Attr.LB, 0) 
    model.optimize()
    no_horn = (model.status == GRB.OPTIMAL)

    print("Peut-on déduire que la licorne a une corne ?", "Oui" if has_horn else "Non")
    print("Peut-on déduire qu'elle n'a pas de corne ?", "Oui" if no_horn else "Non")
else:
    print("La base de connaissances est incohérente.")


Restricted license - for non-production use only - expires 2026-11-23
La base de connaissances est cohérente.
Un modèle trouvé (monde peuplé de licornes) :
Mythique: 0, Mortelle: 1, Mammifère: 1, Corne: 1, Magique: 1
Peut-on déduire que la licorne a une corne ? Oui
Peut-on déduire qu'elle n'a pas de corne ? Oui


## 2 - Gophersat

In [ ]:
import subprocess

# Écrire le fichier DIMACS pour le problème de la licorne
with open("licorne.cnf", "w") as f:
    f.write("p cnf 5 6\n")
    f.write("-1 -2 0\n")
    f.write("1 2 0\n")
    f.write("1 3 0\n")
    f.write("-3 4 0\n")
    f.write("2 4 0\n")
    f.write("-4 5 0\n")

# Exécuter Gophersat avec subprocess
try:
    result = subprocess.run(
        ["../gophersat", "licorne.cnf"], 
        text=True, capture_output=True, check=True
    )
    # Afficher le résultat brut
    print("Résultat brut de Gophersat :")
    print(result.stdout)

    # Analyser le résultat pour voir si le problème est satisfaisable
    if "s SATISFIABLE" in result.stdout:
        print("La base de connaissances est cohérente.")
        # Extraire le modèle si satisfaisable
        start_index = result.stdout.find("v ")
        if start_index != -1:
            model = result.stdout[start_index:].strip()
            print("Un modèle trouvé :")
            print(model)
    else:
        print("La base de connaissances est incohérente.")
except subprocess.CalledProcessError as e:
    print("Erreur lors de l'exécution de Gophersat :", e)

Résultat brut de Gophersat :
c solving licorne.cnf
s SATISFIABLE
v -1 2 3 4 5 0

La base de connaissances est cohérente.
Un modèle trouvé :
v -1 2 3 4 5 0


# Problème II - Graphes